In [1]:
import findspark 
from pyspark.sql import SparkSession
from pyspark.sql import SQLContext  
from pyspark.sql.types import StringType
from pyspark.sql.functions import col, udf
from pyspark.sql.functions import *

In [2]:
findspark.init()

In [3]:
spark_url = 'local'

spark = SparkSession.builder\
        .master(spark_url)\
        .appName('Spark Tutorial')\
        .config('spark.ui.port', '4040')\
        .getOrCreate()

In [4]:
path = 'bangkok_traffy.csv'
df = spark.read.csv(path, header=True, inferSchema=True, sep=',', encoding='utf-8', multiLine=True)

In [5]:
df.show(10, truncate=False)

+-----------+-----------------------------------+----------------------------+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+-------------------------------------------------------------------------------------------------------------------+---------------------------------------------

In [6]:
df.count()

257677

In [7]:

df = df.withColumn('type',regexp_replace('type', r'\{|\}', ''))

In [8]:
#explode type
df = df.withColumn('type',explode(split('type', ',')))

In [9]:
#blank space to ''
df = df.withColumn('type',regexp_replace('type', r'\s+', ''))
#'' to null
df = df.withColumn('type',when(col('type') == '', None).otherwise(col('type')))
df.show()


+-----------+-----------+--------------------+--------------------+--------------------+--------------------+------------------+--------------------+--------------+---------+--------------------+--------------------+--------------+----+------------+--------------------+
|  ticket_id|       type|        organization|             comment|               photo|         photo_after|            coords|             address|   subdistrict| district|            province|           timestamp|         state|star|count_reopen|       last_activity|
+-----------+-----------+--------------------+--------------------+--------------------+--------------------+------------------+--------------------+--------------+---------+--------------------+--------------------+--------------+----+------------+--------------------+
|2021-9LHDM6|       null|                null|            ไม่มีภาพ|https://storage.g...|                null|100.48661,13.79386|1867 จรัญสนิทวงศ์...|       บางพลัด|  บางพลัด|       กรุงเท

In [10]:
df.count()

343013

In [11]:
pd_df = df.toPandas()

In [18]:
pd_df.groupby('type').count().sort_values(by='ticket_id', ascending=False).index.tolist()

['ถนน',
 'ทางเท้า',
 'แสงสว่าง',
 'ความปลอดภัย',
 'น้ำท่วม',
 'ความสะอาด',
 'กีดขวาง',
 'ท่อระบายน้ำ',
 'สะพาน',
 'จราจร',
 'สายไฟ',
 'คลอง',
 'เสียงรบกวน',
 'ต้นไม้',
 'ร้องเรียน',
 'ป้าย',
 'สัตว์จรจัด',
 'PM2.5',
 'สอบถาม',
 'เสนอแนะ',
 'คนจรจัด',
 'การเดินทาง',
 'ห้องน้ำ',
 'ป้ายจราจร',
 '000บาท""ทำให้1.ทางเดินเท้าติดขัดเมื่อมีคนหยุดซื้อของ2.กินที่ทางเดินเท้าจนต้องเดินเบียดกัน3.มีขยะที่เกิดจากทางร้านที่มาตั้งแผง4.ไม่เป็นระเบียบ"',
 'ไล่ฟันวินมอไซด์มีหลายๆคนมาเม้นตามเพจและเฟสบุ๊คส่วนตัวไอจีส่วนตัวช่องยูทูปDahighในแง่ลบในtiktokและสื่อโซเชียลต่างๆก็ลงข่าวทำลายชื่อเสียงwhoopเต็มไปหมดและมีเฟสบุ๊คแอคเคาท์นึงลงรูปตามหาผมกับพี่เจมส์และเม้นท์กัน(ในเชิงลบ)มีการทักมาในinboxแบบเชิงแสดงความเกลียดชังผมจึงพอเข้าใจสถานการณ์ว่าคนส่วนมากที่มากันเมื่อวานเขาคงคิดว่าพวกผมหาเรื่องป้าและรุมฟันวินอย่างที่ข่าวออกผมเข้าใจดีว่าเป็นการเข้าใจผิดของชาวบ้านและเด็กบ้านตรงนั้นและจนถึงตอนนี้ผมได้ทำการย้ายที่อยู่และไม่มีความคิดที่จะกลับไปในพื้นที่แถวนั้นอีกเพราะเพื่อความปลอดภัยของผมและของทุกคนและผมไม่ได้ถือโทษคนที่เ

In [19]:
type = ['ถนน',
 'ทางเท้า',
 'แสงสว่าง',
 'ความปลอดภัย',
 'น้ำท่วม',
 'ความสะอาด',
 'กีดขวาง',
 'ท่อระบายน้ำ',
 'สะพาน',
 'จราจร',
 'สายไฟ',
 'คลอง',
 'เสียงรบกวน',
 'ต้นไม้',
 'ร้องเรียน',
 'ป้าย',
 'สัตว์จรจัด',
 'PM2.5',
 'สอบถาม',
 'เสนอแนะ',
 'คนจรจัด',
 'การเดินทาง',
 'ห้องน้ำ',
 'ป้ายจราจร']

In [23]:
pd_df['is_type'] = pd_df['type'].apply(lambda x: 1 if (x in type) or x is None else 0)

In [25]:
pd_df.drop(pd_df[pd_df['is_type'] == 0].index, inplace=True)

In [27]:
pd_df.shape

(342451, 17)